In [152]:
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
import random

In [153]:
# Read the graph
graph = ig.Graph.Read_Edgelist('C:/3rd quarter/ECE 232/Project2/facebook_combined.txt', directed=False)

In [154]:
# Personalized network of Node 415
core_node = 414
ego_network_vertices = graph.neighborhood(core_node, order=1)
ego_network = graph.subgraph(ego_network_vertices)

In [155]:
# Find the points that have degree of 24
# Calculate the degrees for all vertices
degrees = ego_network.degree()
# Find the vertices with a degree of 24
vertices_with_degree_24 = [v for v, degree in enumerate(degrees) if degree == 24]
# Print the vertices with a degree of 24
print("Vertices with degree 24:", vertices_with_degree_24)
print(ego_network.summary())

Vertices with degree 24: [30, 52, 74, 89, 92, 101, 117, 132, 133, 135, 136]
IGRAPH U--- 160 1857 -- 


In [156]:
# Remove the edge of a given node by 0.25 chance
# Define the probability to remove an edge
prob_remove_edge = 0.25
# Specify the given node ID (assuming the node exists in the graph)
i = 74
# for i in vertices_with_degree_24:
# Get the neighbors for the given node
Neighbors = ego_network.neighbors(i)
# Create a copy of the graph
network = ego_network.copy()
# Initialize the count of deleted edges
R = 0
# Store the deleted edges (using vertex IDs)
deleted_edges = []
# Iterate through the neighbors
for neighbor in Neighbors:
    # Remove the edge with 0.25 chance
    if random.random() < prob_remove_edge:
        deleted_edges.append(neighbor)
        network.delete_edges((i, neighbor))  # Delete the edge using vertex IDs
        R += 1  # Increment the count of deleted edges

# Common Neighbors recommendation (Nodes that have the most mutual friends)
# Get the neighbors (friends) of the specific node
neighbors = network.neighbors(i)
# Get the non-neighbors of the specific node
non_neighbors = list(set(range(network.vcount())) - set(neighbors) - {i})
# Calculate the number of mutual friends between the specific node and its non-neighbors
mutual_friends_count = [len(set(network.neighbors(non_neighbor)).intersection(neighbors)) for non_neighbor in non_neighbors]
# Sort the mutual friends count in descending order and select the top R values
top_R_mutual_friends_count = sorted(mutual_friends_count, reverse=True)[:R]
# Find the indices of the nodes with the top R highest mutual friend counts
top_R_mutual_friends_indices = [k for k, count in enumerate(mutual_friends_count) if count in top_R_mutual_friends_count]
# Get the node IDs with the top R highest mutual friend counts
nodes_with_top_R_mutual_friends = [non_neighbors[k] for k in top_R_mutual_friends_indices]
# Accurancy
result_CN = [number in deleted_edges for number in nodes_with_top_R_mutual_friends]
Accurancy_CN = np.sum(result_CN)/R

# Jaccard measure
# calculate the total neighbours of 2 nodes
total_friends_count = [len(set(network.neighbors(non_neighbor)).union(neighbors)) for non_neighbor in non_neighbors]
Jaccard = np.array(mutual_friends_count)/np.array(total_friends_count)
top_R_Jaccard = sorted(Jaccard, reverse=True)[:R]
top_R_Jaccard_indices = [j for j, count in enumerate(Jaccard) if count in top_R_Jaccard]
nodes_with_top_R_Jaccard = [non_neighbors[j] for j in top_R_Jaccard_indices]
# Accurancy
result_JM = [number in deleted_edges for number in nodes_with_top_R_Jaccard]
Accurancy_JM = np.sum(result_JM)/R

# AAM - Adamic Adar Measure
AAM = []
# Find all mutual friends
for non_neighbor in non_neighbors:
    mutual_friends = set(network.neighbors(non_neighbor)).intersection(neighbors)
    if not mutual_friends:
        AA = 0
    else:
# Get all the degrees
        degrees = network.degree(mutual_friends)
# SUM(1/log(degree))
        AA = np.sum(1/np.log(degrees))
    AAM.append(AA)
top_R_AAM = sorted(AAM, reverse=True)[:R]
top_R_AAM_indices = [j for j, count in enumerate(AAM) if count in top_R_AAM]
nodes_with_top_R_AAM = [non_neighbors[j] for j in top_R_AAM_indices]
# Accurancy
result_AAM = [number in deleted_edges for number in nodes_with_top_R_AAM]
Accurancy_AAM = np.sum(result_AAM)/R

# Print the number of deleted edges and their vertex IDs
print(f"Number of edges deleted: {R}")
print(f"Deleted edges (using vertex IDs): {deleted_edges}")
# Print the node IDs with the top R highest mutual friend counts
print(f"Nodes with the top R highest mutual friend counts (not neighbors) with node {i}: {nodes_with_top_R_mutual_friends}")
print(f"Nodes with the top R highest Jaccard (not neighbors) with node {i}: {nodes_with_top_R_Jaccard}")
print(f"Nodes with the top R highest AAM (not neighbors) with node {i}: {nodes_with_top_R_AAM}")
# Print the accurancy of different measures
print(f"Accurancy of common neighbours:{Accurancy_CN}")
print(f"Accurancy of Jaccard measure:{Accurancy_JM}")
print(f"Accurancy of AAM measure:{Accurancy_AAM}")

Number of edges deleted: 7
Deleted edges (using vertex IDs): [15, 101, 102, 109, 117, 133, 155]
Nodes with the top R highest mutual friend counts (not neighbors) with node 74: [15, 101, 109, 117, 121, 133, 155]
Nodes with the top R highest Jaccard (not neighbors) with node 74: [101, 102, 109, 117, 121, 133, 155]
Nodes with the top R highest AAM (not neighbors) with node 74: [15, 101, 109, 117, 121, 133, 155]
Accurancy of common neighbours:0.8571428571428571
Accurancy of Jaccard measure:0.8571428571428571
Accurancy of AAM measure:0.8571428571428571


In [157]:
# Remove the edge of a given node by 0.25 chance
# Define the probability to remove an edge
prob_remove_edge = 0.25
random.seed()
# Specify the given node ID (assuming the node exists in the graph), run 10 times
accurancy_CN = []
accurancy_JM = []
accurancy_AAM = []
for j in range(10):
    for i in vertices_with_degree_24:

        # Get the neighbors for the given node
        Neighbors = ego_network.neighbors(i)
        # Create a copy of the graph
        network = ego_network.copy()
        # Initialize the count of deleted edges
        R = 0
        # Store the deleted edges (using vertex IDs)
        deleted_edges = []
        # Iterate through the neighbors
        for neighbor in Neighbors:
            # Remove the edge with 0.25 chance
            if random.random() < prob_remove_edge:
                deleted_edges.append(neighbor)
                network.delete_edges((i, neighbor))  # Delete the edge using vertex IDs
                R += 1  # Increment the count of deleted edges

        # Common Neighbors recommendation (Nodes that have the most mutual friends)
        # Get the neighbors (friends) of the specific node
        neighbors = network.neighbors(i)
        # Get the non-neighbors of the specific node
        non_neighbors = list(set(range(network.vcount())) - set(neighbors) - {i})
        # Calculate the number of mutual friends between the specific node and its non-neighbors
        mutual_friends_count = [len(set(network.neighbors(non_neighbor)).intersection(neighbors)) for non_neighbor in non_neighbors]
        # Sort the mutual friends count in descending order and select the top R values
        top_R_mutual_friends_count = sorted(mutual_friends_count, reverse=True)[:R]
        # Find the indices of the nodes with the top R highest mutual friend counts
        top_R_mutual_friends_indices = [k for k, count in enumerate(mutual_friends_count) if count in top_R_mutual_friends_count]
        # Get the node IDs with the top R highest mutual friend counts
        nodes_with_top_R_mutual_friends = [non_neighbors[k] for k in top_R_mutual_friends_indices]
        # Accurancy
        result_CN = [number in deleted_edges for number in nodes_with_top_R_mutual_friends]
        Accurancy_CN = np.sum(result_CN)/R

        # Jaccard measure
        # calculate the total neighbours of 2 nodes
        total_friends_count = [len(set(network.neighbors(non_neighbor)).union(neighbors)) for non_neighbor in non_neighbors]
        Jaccard = np.array(mutual_friends_count)/np.array(total_friends_count)
        top_R_Jaccard = sorted(Jaccard, reverse=True)[:R]
        top_R_Jaccard_indices = [j for j, count in enumerate(Jaccard) if count in top_R_Jaccard]
        nodes_with_top_R_Jaccard = [non_neighbors[j] for j in top_R_Jaccard_indices]
        # Accurancy
        result_JM = [number in deleted_edges for number in nodes_with_top_R_Jaccard]
        Accurancy_JM = np.sum(result_JM)/R

        # AAM - Adamic Adar Measure
        AAM = []
        # Find all mutual friends
        for non_neighbor in non_neighbors:
            mutual_friends = set(network.neighbors(non_neighbor)).intersection(neighbors)
            if not mutual_friends:
                AA = 0
            else:
        # Get all the degrees
                degrees = network.degree(mutual_friends)
        # SUM(1/log(degree))
                AA = np.sum(1/np.log(degrees))
            AAM.append(AA)
        top_R_AAM = sorted(AAM, reverse=True)[:R]
        top_R_AAM_indices = [j for j, count in enumerate(AAM) if count in top_R_AAM]
        nodes_with_top_R_AAM = [non_neighbors[j] for j in top_R_AAM_indices]
        # Accurancy
        result_AAM = [number in deleted_edges for number in nodes_with_top_R_AAM]
        Accurancy_AAM = np.sum(result_AAM)/R
        
        accurancy_CN.append(Accurancy_CN)
        accurancy_JM.append(Accurancy_JM)
        accurancy_AAM.append(Accurancy_AAM)

In [158]:
print(np.average(np.array(accurancy_CN)))
print(np.average(np.array(accurancy_JM)))
print(np.average(np.array(accurancy_AAM)))

0.8505325987144169
0.809013183785911
0.833126393808212
